# matching models solved by linear programming

In [ ]:
# SETUP
using JuMP
using GLPK
using Random
using LinearAlgebra
using Distributions
include("static_functions.jl")

solve_model (generic function with 3 methods)

we start with deterministic many-to-many matching models

the solve_model function is defined such that a matching is found to maximize total surplus (under transferable utilities) and satisfy usual constraints

In [130]:
# deterministic
choice = 2
show_s = true # if we want to print out the match surplus matrix
pr_match = true #if we want to print out match pattern

if choice == 1
    S = repeat(reshape(1:8,:,1),1,4) 
    q = [1,2,3,2] #quota for each firm 
    println("8 workers with increasing ability sorted into 4 firms with 8 spots")
    solve_model(S, pr_match)
elseif choice == 2
    S = repeat(reshape(1:12,:,1),1,4) 
    println("12 workers with increasing ability sorted into 4 firms with 8 spots")
    solve_model(S, pr_match)
elseif choice == 3
    S = float(repeat(reshape(1:12,:,1),1,4))
    S[:,1] .+= 0.51 
    S[:,2] .+= 0.5
    q = [3,1,4,2]
    println("12 workers into 4 firms/8 jobs with different firm productivity")
    solve_model(S, pr_match,q)
end 

show_s && display(S)

# notice how in case 3 most productive firms choose first from descending order

12×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0

12 workers with increasing ability sorted into 4 firms with 8 spots
Worker values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Firm values:   [8.0, 8.0, 8.0, 8.0]


12×4 Matrix{Int64}:
  1   1   1   1
  2   2   2   2
  3   3   3   3
  4   4   4   4
  5   5   5   5
  6   6   6   6
  7   7   7   7
  8   8   8   8
  9   9   9   9
 10  10  10  10
 11  11  11  11
 12  12  12  12

we add components to allow for signaling 

In [131]:
noise_signal = 5
pr_match = false
q = [3,2,1,4]

ability = float(repeat(reshape(1:12,:,1),1,4))
ability[:,1] .+= 5
ability[:,4] .-= 1
# display(ability)
opt_match = solve_model(ability, pr_match)
optimal = sum(ability .* opt_match)

signal = ability .+ rand(Normal(0, noise_signal), size(ability))
noise_match = solve_model(signal, pr_match)

total_surplus = sum(ability .* noise_match)
println("optimal match value ", optimal, " and noised match value ", total_surplus)


Worker values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0]
Firm values:   [13.0, 8.0, 8.0, 7.0]
Worker values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.625, 0.0, 5.263, 0.0]
Firm values:   [20.548, 14.142, 16.284, 6.994]
optimal match value 46.0 and noised match value 46.0


adding some components to allow for complementarity 
assuming workers and firms of two types, we add a compatibility term into the surplus matrix

In [ ]:
function matching(no_w, no_f, compat_term, q)   
    compat_term = 0.5
    ability = float(repeat(reshape(1:no_w,:,1),1,no_f))
    # display(ability)
    type_w = rand(0:1, no_w)
    type_f = rand(0:1, no_f)
    for i in 1:no_w, j in 1:no_f
        if type_w[i] == type_f[j]
            ability[i,j] += compat_term
        end
    end
    display(ability)
    # q = fill(1, no_f)
    # match = solve_model(ability, pr_match, q)
    # total_matches, same_type_matches = 0, 0
    # for i in 1:no_w, j in 1:no_f
    #     if match[i, j] > 0.5  # matched
    #         total_matches += 1
    #         if type_w[i] == type_f[j]
    #             same_type_matches += 1
    #         end
    #     end
    # end
    # prop_same = same_type_matches / total_matches
    # # print(same_type_matches, total_matches)
    # return prop_same
end

matching(15,15, -6005)


# plot out to see that as complementarity term change 
# the proportion of same-type match changes accordingly

Worker values: [1.0, 2.0, 3.0, 4.5, 5.0, 6.0, 7.5, 8.0, 9.5, 10.0, 11.0, 12.0, 13.0, 14.5, 15.0]
Firm values:   [0.5, -0.0, -0.0, 0.5, 0.5, 0.5, 0.5, -0.0, -0.0, -0.0, 0.5, -0.0, -0.0, 0.5, 0.0]
1115

0.7333333333333333